In [1]:
from pymilvus import MilvusClient

#数据库名称
db_name = "cost_assistant_sh"
#集合名称  一个数据库内可能有多个不同名的集合 集合可以和数据库同名
cl_name = "cost_assistant_sh"


In [ ]:
# 加载或创建现有数据
# client = MilvusClient("milvus_demo.db")

# 加载远程数据 连接到Milvus服务器，这里假设服务器本地运行，端口默认19530，若实际情况不同需修改 
# 建议这里就直接定位到需要的数据库
client = MilvusClient(uri="http://10.193.41.29", port="19530",db_name=db_name) 
# 列出所有数据库，确认目标数据库是否存在
databases = client.list_databases() 
print( "数据库的数量:",len(databases)) #数据库的数量
print( "现有数据库如下:")  
for database in databases:
    print(database ) 

数据库的数量: 7
现有数据库如下:
default
sepd_vector_store
cost_assistant_sh_BM25
cost_assistant_sh
cost_assistant_gw_BM25
cost_assistant_gw
intention


In [3]:

# 遍历每个collection名称并获取对应的collection对象
for database in databases:
    print(database ) 
    if(database==db_name):
        collections = client.list_collections()
        print(len(collections))
        break

print(len(collections))
# for collection in collections :
#     print(collection) 
collection =collections[0]
print(collection)

intention
default
sepd_vector_store
cost_assistant_sh_BM25
cost_assistant_sh
1
1
cost_assistant_sh


In [28]:
# 查询集合中的数据
collection_name = cl_name # 替换为你的集合名称
filter_condition = 'name_property == "该设备为：500kV变压器，属于：一次设备，规格为：500kV,334MVA,单相,500/220/66,无励磁,三绕组自耦"'  # 替换为你的过滤条件
output_fields = ["id","name_property" ,"describ_dense"]  # 替换为你的字段名称

# 执行查询
res = client.query(
    collection_name=collection_name,
    filter=filter_condition,
    output_fields=output_fields,
    limit=3  # 限制返回的结果数量
)

# 打印查询结果
print(res)

data: ["{'id': 5, 'name_property': '该设备为：500kV变压器，属于：一次设备，规格为：500kV,334MVA,单相,500/220/66,无励磁,三绕组自耦', 'describ_dense': [-0.006548017, 0.023878329, -0.042785533, 0.041701473, 0.021001427, -0.014218672, 0.011657427, 0.08013745, 0.026170133, 0.02300958, 0.004172615, -0.0071427696, -0.0034473957, -0.017091304, -0.00828348, -0.007815264, 0.008189925, -0.030305753, 0.0296191, -0.00038980838, -0.030009044, 0.016146349, 0.008843948, 0.03218753, -0.003568792, 0.0063475696, -0.05444416, -0.012470382, 0.03894844, 0.0075854505, -0.026214778, -0.029688984, 0.023864187, 0.011979978, -0.010803895, -0.022323264, -0.04037944, -0.0047815545, -0.037706945, -0.0038031912, 0.031733934, -0.058795936, 0.020232078, -0.0002788295, 0.037519168, -0.0102744205, -0.05351264, -0.017126713, -0.03040627, -0.03942076, 0.011906786, -0.11580734, 0.04804346, -0.031165566, -0.0041748895, -0.00791451, -0.030669231, -0.051973555, -0.015607957, 0.01036375, -0.04654464, 0.050130904, -0.007935979, -0.012731008, 0.06395367, 0.00

In [ ]:
## 向量查询集合中的数据
vector_field = "describ_dense" #搜索的向量字段
collection_name = cl_name # 替换为你的集合名称 
query_vectors=[res[0][vector_field]]# 用于的查询向量
output_fields = ["name_property" ,vector_field]  # 替换为你的字段名称
top_k = 3

## 查询 
search_results = client.search(
    collection_name=collection_name,  # target collection
    data=query_vectors,  # query vectors
    anns_field=vector_field,  # 指定要搜索的向量字段
    limit=top_k,  # number of returned entities
    output_fields=output_fields,  # specifies fields to be returned
)

print(search_results)  
print( [item['id'] for item in search_results[0]])  
 

data: ["[{'id': 5, 'distance': 0.9999998807907104, 'entity': {'name_property': '该设备为：500kV变压器，属于：一次设备，规格为：500kV,334MVA,单相,500/220/66,无励磁,三绕组自耦', 'describ_dense': [-0.006548016797751188, 0.023878328502178192, -0.042785532772541046, 0.04170147329568863, 0.021001426503062248, -0.014218672178685665, 0.011657427065074444, 0.08013744652271271, 0.02617013268172741, 0.02300957962870598, 0.004172614775598049, -0.007142769638448954, -0.0034473957493901253, -0.017091304063796997, -0.008283480070531368, -0.007815264165401459, 0.008189924992620945, -0.030305752530694008, 0.02961909957230091, -0.0003898083814419806, -0.030009044334292412, 0.016146348789334297, 0.008843948133289814, 0.032187528908252716, -0.0035687920171767473, 0.006347569637000561, -0.05444416031241417, -0.012470382265746593, 0.03894843906164169, 0.007585450541228056, -0.026214778423309326, -0.029688984155654907, 0.02386418730020523, 0.01197997760027647, -0.010803895071148872, -0.022323263809084892, -0.04037943854928017, -0.00478155

# 其他操作

In [ ]:
# 
from pymilvus import MilvusClient 
client = MilvusClient("milvus_demo.db")

# 创建集合
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)
 

In [35]:
# 插入操作
from random import random
from pymilvus import model

# If connection to https://huggingface.co/ failed, uncomment the following path
# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# This will download a small embedding model "paraphrase-albert-small-v2" (~50MB).
embedding_fn = model.DefaultEmbeddingFunction()

# Text strings to search from.
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
# Use fake representation with random vectors (768 dimension).
vectors = [[random.uniform(-1, 1) for _ in range(768)] for _ in docs]
data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))
res = client.insert(collection_name="demo_collection", data=data) 
print(res)

ImportError: The 'milvus_model' package is not installed. For installation, use 'pip install pymilvus[model]'. For more information, please visit https://github.com/milvus-io/milvus-model.

In [ ]:
# Drop collection 删除集合
# client.drop_collection(collection_name="demo_collection")